In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.utils import set_seed, jprint

set_seed(89)

In [3]:
pd.options.display.float_format = '{:,.3f}'.format

In [4]:
from tqdm.auto import tqdm
tqdm.pandas()

In [5]:
def load_result_dataframe(pattern: str = "our-method-kgqa-report-20240819*.jsonl") -> pd.DataFrame:
    dfs = [pd.read_json(filename, orient='records', lines=True) for filename in Path("./data").glob(pattern)]

    # for run, df in enumerate(dfs):
    #     df["run"] = run + 1

    return pd.concat(dfs, ignore_index=True) 

In [6]:
raw_df = load_result_dataframe()
raw_df.rename(
    columns={
        'qdecomp': 'Q-decomp',
        'context': 'Context',
        'retrieval': 'Retrieval',
        'top_k': 'Top-k',
        'qa': 'QA Prompting',
        'run': 'Run',
        'exact_match': "EM",
        'f1': "F1",
    }, 
    inplace=True,
)
raw_df

,Q-decomp,Context,Retrieval,Top-k,QA Prompting,Run,EM,F1
0,False,Triplets,Sparse,30,Standard,1,0.270,0.407
1,False,Triplets,Sparse,40,Standard,1,0.240,0.378
2,False,Triplets,Sparse,50,Standard,1,0.240,0.365
3,False,Triplets,Dense,30,Standard,1,0.210,0.339
4,False,Triplets,Dense,40,Standard,1,0.210,0.361
...,...,...,...,...,...,...,...,...
115,True,Triplets,Dense,100,Standard,1,0.410,0.508
116,False,Triplets,Sparse,100,Standard,2,0.220,0.345
117,False,Triplets,Dense,100,Standard,2,0.190,0.315
118,True,Triplets,Sparse,100,Standard,2,0.360,0.449


In [7]:
mask = (raw_df['Retrieval'].isin(['Dense', 'Sparse'])) & raw_df['QA Prompting'].isin(['Standard'])
df = raw_df.loc[mask].copy().drop(columns=['Context', 'QA Prompting'])
# df = raw_df.copy().drop(columns=['Context', 'QA Prompting'])

In [8]:
with open("experiment-results-detailed-kgqa.tex", 'w') as f:
    f.write(df.sort_values(['Q-decomp', 'Retrieval', 'Top-k', 'Run']).to_latex(index=False, float_format='%.3f'))

In [9]:
agg_df = df.drop(columns=['Run']).groupby(['Q-decomp', 'Retrieval', 'Top-k']).agg(['min', 'mean', 'max', 'std'])
agg_df

EM                      F1                  
                           min  mean   max   std   min  mean   max   std
Q-decomp Retrieval Top-k                                                
False    Dense     5     0.200 0.203 0.210 0.006 0.312 0.314 0.316 0.002
                   10    0.150 0.157 0.160 0.006 0.263 0.268 0.274 0.006
                   15    0.180 0.190 0.200 0.010 0.297 0.312 0.329 0.016
                   20    0.210 0.213 0.220 0.006 0.329 0.334 0.340 0.006
                   30    0.210 0.210 0.210 0.000 0.333 0.338 0.343 0.005
                   40    0.210 0.220 0.230 0.010 0.357 0.364 0.375 0.010
                   50    0.210 0.223 0.240 0.015 0.333 0.349 0.368 0.018
                   70    0.190 0.200 0.210 0.010 0.324 0.328 0.332 0.004
                   100   0.190 0.210 0.220 0.017 0.315 0.335 0.346 0.017
         Sparse    5     0.210 0.220 0.230 0.010 0.320 0.326 0.331 0.006
                   10    0.230 0.237 0.240 0.006 0.347 0.357 0.370 0.012
                   15    0.240 0.250 0.260 0.010 0.360 0.375 0.392 0.016
                   20    0.230 0.230 0.230 0.000 0.353 0.356 0.363 0.006
                   30    0.270 0.277 0.290 0.012 0.407 0.416 0.427 0.010
                   40    0.240 0.247 0.250 0.006 0.378 0.381 0.385 0.004
                   50    0.240 0.253 0.270 0.015 0.365 0.378 0.395 0.015
                   70    0.180 0.180 0.180 0.000 0.272 0.275 0.277 0.003
                   100   0.200 0.210 0.220 0.010 0.324 0.333 0.345 0.011
True     Dense     5     0.370 0.380 0.390 0.010 0.452 0.457 0.464 0.006
                   10    0.390 0.393 0.400 0.006 0.469 0.471 0.474 0.003
                   15    0.410 0.417 0.420 0.006 0.486 0.493 0.500 0.007
                   20    0.410 0.430 0.440 0.017 0.499 0.514 0.531 0.016
                   30    0.430 0.437 0.440 0.006 0.505 0.511 0.515 0.005
                   40    0.410 0.417 0.430 0.012 0.475 0.489 0.500 0.013
                   50    0.440 0.447 0.460 0.012 0.516 0.529 0.553 0.020
                   70    0.420 0.433 0.450 0.015 0.518 0.530 0.544 0.013
                   100   0.400 0.417 0.440 0.021 0.499 0.509 0.522 0.012
         Sparse    5     0.270 0.280 0.290 0.010 0.351 0.358 0.362 0.006
                   10    0.340 0.350 0.360 0.010 0.448 0.451 0.453 0.003
                   15    0.360 0.370 0.380 0.010 0.455 0.472 0.490 0.018
                   20    0.390 0.397 0.400 0.006 0.494 0.502 0.515 0.011
                   30    0.400 0.407 0.410 0.006 0.499 0.505 0.514 0.008
                   40    0.390 0.400 0.410 0.010 0.498 0.500 0.502 0.002
                   50    0.360 0.373 0.390 0.015 0.459 0.468 0.484 0.014
                   70    0.340 0.347 0.350 0.006 0.421 0.424 0.426 0.003
                   100   0.360 0.367 0.380 0.012 0.449 0.456 0.462 0.007

In [19]:
mask = (raw_df['Retrieval'].isin(['Dense', 'Sparse'])) & (raw_df['QA Prompting'].isin(['Standard'])) 
# & (raw_df['Top-k'].isin([10, 20, 40, 70, 100]))
kgqa_df = raw_df.loc[mask].copy().sort_values(['Q-decomp', 'Retrieval', 'Top-k', 'Run'])

In [20]:
with open("table-kgqa-all.tex", 'w') as f:
    f.write(kgqa_df.drop(columns=['Context', 'QA Prompting']).to_latex(index=False, float_format='%.3f'))

kgqa_df

,Q-decomp,Context,Retrieval,Top-k,QA Prompting,Run,EM,F1
60,False,Triplets,Dense,5,Standard,1,0.210,0.315
80,False,Triplets,Dense,5,Standard,2,0.200,0.316
40,False,Triplets,Dense,5,Standard,3,0.200,0.312
61,False,Triplets,Dense,10,Standard,1,0.160,0.268
81,False,Triplets,Dense,10,Standard,2,0.150,0.263
...,...,...,...,...,...,...,...,...
102,True,Triplets,Sparse,70,Standard,2,0.340,0.421
106,True,Triplets,Sparse,70,Standard,3,0.350,0.425
114,True,Triplets,Sparse,100,Standard,1,0.380,0.462
118,True,Triplets,Sparse,100,Standard,2,0.360,0.449


In [11]:
kgqa_agg_df = kgqa_df.drop(columns=['Context', 'QA Prompting']).drop(columns=['Run']).groupby(['Q-decomp', 'Retrieval', 'Top-k']).mean().sort_values(['Q-decomp', 'Retrieval', 'Top-k'])

with open("table-kgqa-agg.tex", 'w') as f:
    f.write(kgqa_agg_df.to_latex(float_format='%.3f'))

kgqa_agg_df

EM    F1
Q-decomp Retrieval Top-k            
False    Dense     5     0.203 0.314
                   10    0.157 0.268
                   15    0.190 0.312
                   20    0.213 0.334
                   30    0.210 0.338
                   40    0.220 0.364
                   50    0.223 0.349
                   70    0.200 0.328
                   100   0.210 0.335
         Sparse    5     0.220 0.326
                   10    0.237 0.357
                   15    0.250 0.375
                   20    0.230 0.356
                   30    0.277 0.416
                   40    0.247 0.381
                   50    0.253 0.378
                   70    0.180 0.275
                   100   0.210 0.333
True     Dense     5     0.380 0.457
                   10    0.393 0.471
                   15    0.417 0.493
                   20    0.430 0.514
                   30    0.437 0.511
                   40    0.417 0.489
                   50    0.447 0.529
                   70    0.433 0.530
                   100   0.417 0.509
         Sparse    5     0.280 0.358
                   10    0.350 0.451
                   15    0.370 0.472
                   20    0.397 0.502
                   30    0.407 0.505
                   40    0.400 0.500
                   50    0.373 0.468
                   70    0.347 0.424
                   100   0.367 0.456

## Inspect

In [11]:
agg_df.sort_values('F1', ascending=False)

EM    F1
Q-decomp Retrieval Top-k            
True     Dense     50    0.447 0.529
                   20    0.430 0.514
                   30    0.437 0.511
         Sparse    30    0.407 0.505
                   20    0.397 0.502
                   40    0.400 0.500
         Dense     15    0.417 0.493
                   40    0.417 0.489
         Sparse    15    0.370 0.472
         Dense     10    0.393 0.471
         Sparse    50    0.373 0.468
         Dense     5     0.380 0.457
         Sparse    10    0.350 0.451
False    Sparse    30    0.277 0.416
                   40    0.247 0.381
                   50    0.253 0.378
                   15    0.250 0.375
         Dense     40    0.220 0.364
True     Sparse    5     0.280 0.358
False    Sparse    10    0.237 0.357
                   20    0.230 0.356
         Dense     50    0.223 0.349
                   30    0.210 0.338
                   20    0.213 0.334
         Sparse    5     0.220 0.326
         Dense     5     0.203 0.314
                   15    0.190 0.312
                   10    0.157 0.268

In [12]:
idf = agg_df.reset_index()
idf.head()

,Q-decomp,Retrieval,Top-k,EM,F1
0,False,Dense,5,0.203,0.314
1,False,Dense,10,0.157,0.268
2,False,Dense,15,0.190,0.312
3,False,Dense,20,0.213,0.334
4,False,Dense,30,0.210,0.338


In [13]:
qd_mask = idf['Q-decomp']
retrieval_mask = idf['Retrieval'].isin(['Dummy', 'Sparse', 'Dense', 'Perfect'])
idf.loc[retrieval_mask].sort_values('F1', ascending=False)

,Q-decomp,Retrieval,Top-k,EM,F1
20,True,Dense,50,0.447,0.529
17,True,Dense,20,0.430,0.514
18,True,Dense,30,0.437,0.511
25,True,Sparse,30,0.407,0.505
24,True,Sparse,20,0.397,0.502
26,True,Sparse,40,0.400,0.500
16,True,Dense,15,0.417,0.493
19,True,Dense,40,0.417,0.489
23,True,Sparse,15,0.370,0.472
15,True,Dense,10,0.393,0.471
